# **Week 5 - Capstone Development**
## **Support Vector Machines**

Ok so for this one, we will take the best performing dataset in the weeks leading up to this, and then check out how a support vector 'classifier'  


Source: https://scikit-learn.org/stable/modules/svm.html

### **Imports**

In [4]:
# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns
import matplotlib.pyplot as plt



# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedStratifiedKFold,
    RepeatedKFold
)

from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, root_mean_squared_error, accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_predict, KFold
# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

### **Dataset Imports**

In [6]:
# Base Datasets
BDB_All_Plays_Model_Ready = pd.read_csv("../../BDB_All_Plays_Model_Ready.csv") # Big Data Bowl Dataset
PDA_Model_Ready = pd.read_csv("../../PDA_Model_Ready.csv") # Punt Data Analytics
FNF_Model_Ready = pd.read_csv("../../FNF_Model_Ready.csv") # First and Future


# PCA and Standardized Datasets
PDA_PCA_Features = pd.read_csv('../../Feature_Subsets/PDA_PCA_Features.csv')
FNF_PCA_Features = pd.read_csv('../../Feature_Subsets/FNF_PCA_Features.csv')
BDB_PCA_Features = pd.read_csv('../../Feature_Subsets/BDB_PCA_Features.csv')

### **Useful Functions**

Here we will import functions used in previous weeks to handle our data modeling. 

#### **Train Test Split**

In [7]:
# ===========================================================================================
# Function taken from Module 3 Final Project
# https://github.com/LeeMcFarling/Final_Project_Writeup/blob/main/Final_Project_Report.ipynb
# ===========================================================================================

def train_test_split_data(df, target_col):
    X = df.drop(columns=target_col)
    y = df[target_col]
# 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

And used on the data: 

In [9]:
PDA_PCA_X_train, PDA_PCA_X_test, PDA_PCA_y_train, PDA_PCA_y_test = train_test_split_data(PDA_PCA_Features, 'Inj_Occured')
FNF_PCA_X_train, FNF_PCA_X_test, FNF_PCA_y_train, FNF_PCA_y_test = train_test_split_data(FNF_PCA_Features, 'Inj_Occured')
BDB_PCA_X_train, BDB_PCA_X_test, BDB_PCA_y_train, BDB_PCA_y_test = train_test_split_data(BDB_PCA_Features, 'Inj_Occured')

bdb_X_train, bdb_X_test, bdb_y_train, bdb_y_test = train_test_split_data(BDB_All_Plays_Model_Ready, 'Inj_Occured')
fnf_X_train, fnf_X_test, fnf_y_train, fnf_y_test = train_test_split_data(FNF_Model_Ready, 'Inj_Occured')
pda_X_train, pda_X_test, pda_y_train, pda_y_test = train_test_split_data(PDA_Model_Ready, 'Inj_Occured')


#### **Standardization Function**

In [10]:
# =============================================================================================
# Standardize Numeric Features
#
# Taken from Week 2.ipynb and modified to exclude the hard coded logic
#  
# Originally taken from Mod 3 final project found here: 
# https://github.com/LeeMcFarling/Final_Project_Writeup/blob/main/Final_Project_Report.ipynb
#
# =============================================================================================


def standardize_features(df, target_column=None, debug=False, return_scaler=False):
    df_scaled = df.copy()

    # 1) Automatically detect numeric columns
    numeric_cols = df.select_dtypes(include="number").columns.tolist()

    # 2) Remove target if present
    if target_column in numeric_cols:
        numeric_cols.remove(target_column)

    # 3) Exclude binary (0/1) columns
    numeric_cols_to_scale = [
        col for col in numeric_cols
        if df[col].nunique(dropna=True) > 2
    ]

    if debug:
        print("Numeric columns before filtering:", numeric_cols)
        print("Numeric columns after filtering:", numeric_cols_to_scale)

    # 4) Apply StandardScaler
    scaler = StandardScaler()
    if numeric_cols_to_scale:
        df_scaled[numeric_cols_to_scale] = scaler.fit_transform(df_scaled[numeric_cols_to_scale])

    if return_scaler:
        return df_scaled, scaler
    return df_scaled

And then we'll scale our feature sets. 

In [11]:
PDA_PCA_X_train_scaled = standardize_features(PDA_PCA_X_train)
PDA_PCA_X_test_scaled = standardize_features(PDA_PCA_X_test)
# PDA_PCA_y_train
# PDA_PCA_y_test 


FNF_PCA_X_train_scaled = standardize_features(FNF_PCA_X_train) 
FNF_PCA_X_test_scaled = standardize_features(FNF_PCA_X_test)
# FNF_PCA_y_train
# FNF_PCA_y_test 

BDB_PCA_X_train_scaled = standardize_features(BDB_PCA_X_train)
BDB_PCA_X_test_scaled  = standardize_features(BDB_PCA_X_test)
# BDB_PCA_y_train
# BDB_PCA_y_test 

bdb_X_train_scaled = standardize_features(bdb_X_train)
bdb_X_test_scaled = standardize_features(bdb_X_test)
# bdb_y_train
# bdb_y_test 

fnf_X_train_scaled = standardize_features(fnf_X_train)
fnf_X_test_scaled = standardize_features(fnf_X_test)
# fnf_y_train
# fnf_y_test 

pda_X_train_scaled = standardize_features(pda_X_train)
pda_X_test_scaled = standardize_features(pda_X_test)
# pda_y_train
# pda_y_test 

#### **Run Model Classifier**

In [12]:
# =============================================================================================
# Taken from Mod 3 Week 8:
# https://github.com/waysnyder/Module-3-Assignments/blob/main/Homework_08.ipynb
# 
# Global dataframe logic taken from mod 3 final project: 
# https://github.com/LeeMcFarling/Final_Project_Writeup/blob/main/Final_Project_Report.ipynb
# 
# Final Function was developed in Week 2 of this Module
# =============================================================================================

def run_model_classifier(model, X_train, y_train, X_test, y_test, n_repeats=10, n_jobs=-1, run_comment=None, return_model=False, concat_results=False, **model_params):

    global combined_results
    # Remove extra key used to store error metric, if it was added to the parameter dictionary
    if 'accuracy_found' in model_params:
        model_params = model_params.copy()
        model_params.pop('accuracy_found', None)  
        
    # Instantiate the model if a class is provided
    if isinstance(model, type):
        model = model(**model_params)
    else:                                    
        model.set_params(**model_params)    

    model_name = model.__name__ if isinstance(model, type) else model.__class__.__name__ # Added because 


    # Use RepeatedStratifiedKFold for classification to preserve class distribution
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=n_repeats, random_state=42)
    
    # Perform 5-fold cross-validation using accuracy as the scoring metric
    cv_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=n_jobs)
    
    mean_cv_accuracy = np.mean(cv_scores)
    std_cv_accuracy  = np.std(cv_scores)
    
    # Fit the model on the full training set
    model.fit(X_train, y_train)
    
    # Compute training and testing accuracy
    train_preds    = model.predict(X_train)
    test_preds     = model.predict(X_test)

    # Normal Accuracy 
    train_accuracy = accuracy_score(y_train, train_preds)
    test_accuracy  = accuracy_score(y_test, test_preds)

    # Balanced Accuracy Metrics
    balanced_train_accuracy = balanced_accuracy_score(y_train, train_preds)
    balanced_test_accuracy = balanced_accuracy_score(y_test, test_preds)

    results_df = pd.DataFrame([{
        'model': model_name, 
        'model_params': model.get_params(),
        'mean_cv_accuracy': mean_cv_accuracy,
        'std_cv_accuracy': std_cv_accuracy,
        'train_accuracy': train_accuracy, 
        'test_accuracy': test_accuracy,
        'balanced_train_accuracy' : balanced_train_accuracy,
        'balanced_test_accuracy': balanced_test_accuracy,
        'run_comment': run_comment
    }])
    
    if concat_results:
        try:
            combined_results = pd.concat([combined_results, results_df], ignore_index=True)
        except NameError:
            combined_results = results_df

    return (results_df, model) if return_model else results_df

___

## **Modeling**

Let's start with a Linear Support Vector Classifier with parameters set to the same general params as our original Logistic Regression. 

Let's start with the Big Data Bowl Dataset: 

In [16]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

BDB_SVC_results_df = run_model_classifier(
    SVC,
    bdb_X_train_scaled, 
    bdb_y_train, 
    bdb_X_test_scaled,
    bdb_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='BDB - Baseline SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.952888,0.005041,0.979529,0.939181,0.989505,0.51744,BDB - Linear SVC


And then try it on the First and Future Dataset: 

In [18]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

FNF_SVC_results_df = run_model_classifier(
    SVC,
    fnf_X_train_scaled, 
    fnf_y_train, 
    fnf_X_test_scaled,
    fnf_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='FNF - Baseline SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.998537,0.000194,0.99868,0.998408,0.99934,0.499298,FNF - Baseline SVC - rbf kernel


And finally, let's try it on the Punt Data Analytics Dataset. 

In [19]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

PDA_SVC_results_df = run_model_classifier(
    SVC,
    pda_X_train_scaled, 
    pda_y_train, 
    pda_X_test_scaled,
    pda_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA - Baseline SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.960104,0.004415,0.972867,0.959611,0.98624,0.573084,PDA - Baseline SVC - rbf kernel


___

## **PCA Scaled - SVM**

And before we continue onto trying other kernels in the SVM Classifier, let's try plugging in the PCA and Standardized data from week 3 to see how model performance checks out. 

#### **Big Data Bowl**

In [20]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

BDB_PCA_SVC_results_df = run_model_classifier(
    SVC,
    BDB_PCA_X_train_scaled, 
    BDB_PCA_y_train, 
    BDB_PCA_X_test_scaled,
    BDB_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='BDB - PCA Transformed -  SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_PCA_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.953649,0.004447,0.981576,0.94152,0.990555,0.518638,BDB - PCA Transformed - SVC - rbf kernel


#### **First and Future**

In [21]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

FNF_PCA_SVC_results_df = run_model_classifier(
    SVC,
    FNF_PCA_X_train_scaled, 
    FNF_PCA_y_train, 
    FNF_PCA_X_test_scaled,
    FNF_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='FNF - PCA Transformed -  SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_PCA_SVC_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.998547,0.000213,0.998722,0.99824,0.999361,0.499213,FNF - PCA Transformed - SVC - rbf kernel


#### **Punt Data Analytics**

In [22]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0

}

PDA_PCA_SVC_results_df = run_model_classifier(
    SVC,
    PDA_PCA_X_train_scaled, 
    PDA_PCA_y_train, 
    PDA_PCA_X_test_scaled,
    PDA_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA - PCA Transformed -  SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_PCA_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.955951,0.004697,0.969124,0.94914,0.984342,0.567781,PDA - PCA Transformed - SVC - rbf kernel


###

So, this will be discussed more in the discussion part of the analysis, but these are horrifically overfitting. I'll try an experiment with some hyperparameters here. 

___

### **Quick Aside:  to Reduce Overfitting**

In [27]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'rbf', 
    'C': 1.0,
    'gamma': 'auto'

}

PDA_G_PCA_SVC_results_df = run_model_classifier(
    SVC,
    PDA_PCA_X_train_scaled, 
    PDA_PCA_y_train, 
    PDA_PCA_X_test_scaled,
    PDA_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA - Different Gamma Value - PCA Transformed -  SVC - rbf kernel', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_G_PCA_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.671331,0.018914,0.651946,0.653702,0.790637,0.534291,PDA - Different Gamma Value - PCA Transformed ...


Hmm, so it looks like there is less accuracy on the mean_CV which would indicate that the model is overfitting less, but the balanced test accuracy is still barely better than random guessing. 

___

### **Preliminary Results**

In [28]:
combined_results.sort_values(by='balanced_test_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.960104,0.004415,0.972867,0.959611,0.986240,0.573084,PDA - Baseline SVC - rbf kernel
5,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.955951,0.004697,0.969124,0.949140,0.984342,0.567781,PDA - PCA Transformed - SVC - rbf kernel
6,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.671331,0.018914,0.651946,0.653702,0.790637,0.534291,PDA - Different Gamma Value - PCA Transformed ...
3,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.953649,0.004447,0.981576,0.941520,0.990555,0.518638,BDB - PCA Transformed - SVC - rbf kernel
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.952888,0.005041,0.979529,0.939181,0.989505,0.517440,BDB - Linear SVC
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.998537,0.000194,0.998680,0.998408,0.999340,0.499298,FNF - Baseline SVC - rbf kernel
4,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.998547,0.000213,0.998722,0.998240,0.999361,0.499213,FNF - PCA Transformed - SVC - rbf kernel


Oof, so the training accuracy and the mean_cv accuracy metrics are all insanely high (>95%), while the balanced test accuracy metrics are all super low (barely better than random guessing). All things considered, this means that the models are horrifically overfitting on the underlying class imbalance. Because our Injury Rate ranges from about 2.5% and below, even though we set our class_weight to 'balanced', it would seem that the overall 'injured' class is not nearly sufficient for the SVM to pick up on it. Even when a different gamma hyperparameter value was used, the model was still barely better than random guessing.



For the point of Illustration of different kernels, I'll repeat the experiments above on a Linear SVM model to see if that helps, but I don't expect it to help. 

____

### **SVM with a Linear Kernel**

#### **Big Data Bowl Set**

In [29]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'linear', 
    'C': 1.0

}

BDB_lin_SVC_results_df = run_model_classifier(
    SVC,
    bdb_X_train_scaled, 
    bdb_y_train, 
    bdb_X_test_scaled,
    bdb_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='linear kernel - BDB - SVC', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_lin_SVC_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/py

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.66068,0.015038,0.669981,0.650877,0.741417,0.540644,linear kernel - BDB - SVC


#### **First and Future Set**

In [30]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'linear', 
    'C': 1.0

}

FNF_lin_SVC_results_df = run_model_classifier(
    SVC,
    fnf_X_train_scaled, 
    fnf_y_train, 
    fnf_X_test_scaled,
    fnf_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='linear kernel - FNF - SVC', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_lin_SVC_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/py

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.955172,0.033723,0.936298,0.936519,0.505598,0.468347,linear kernel - FNF - SVC


#### **Punt Data Analytics**

In [31]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42,
    'kernel': 'linear', 
    'C': 1.0

}

PDA_lin_SVC_results_df = run_model_classifier(
    SVC,
    pda_X_train_scaled, 
    pda_y_train, 
    pda_X_test_scaled,
    pda_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='linear kernel - PDA - SVC', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_lin_SVC_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.656547,0.017538,0.64259,0.63276,0.759605,0.523685,linear kernel - PDA - SVC


#### **Linear Results**

In [33]:
combined_results[combined_results['run_comment'].str.contains('linear')].sort_values(by='balanced_test_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
7,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.660680,0.015038,0.669981,0.650877,0.741417,0.540644,linear kernel - BDB - SVC
9,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.656547,0.017538,0.642590,0.632760,0.759605,0.523685,linear kernel - PDA - SVC
8,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.955172,0.033723,0.936298,0.936519,0.505598,0.468347,linear kernel - FNF - SVC


Yeah that didn't fair much better than the other ones. 

___